# Conseguimos los datos 

Importamos el dataset de piedra, papel y tijeras a la carpeta temporal:

In [33]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/rps.zip \
    -O /tmp/rps.zip
  


!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/rps-test-set.zip \
    -O /tmp/rps-test-set.zip

--2023-05-10 12:12:18--  https://storage.googleapis.com/learning-datasets/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 142.250.153.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘/tmp/rps.zip’

/tmp/rps.zip        100%[===================>] 191.38M  45.5MB/s    in 4.5s    

2023-05-10 12:12:23 (42.8 MB/s) - ‘/tmp/rps.zip’ saved [200682221/200682221]

--2023-05-10 12:12:23--  https://storage.googleapis.com/learning-datasets/rps-test-set.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 142.250.153.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29516758 (28M) [application/zip]
Saving to: ‘/tmp/rps-test-set.zip’

/

Extraemos los datos y los descomprimimos:



In [34]:

import os
import zipfile

local_zip = '/tmp/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

local_zip = '/tmp/rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

Creamos las carpetas para trabajar de forma organizada y las llenamos con los elementos correspondientes

In [35]:
rock_dir = os.path.join('/tmp/rps/rock')
paper_dir = os.path.join('/tmp/rps/paper')
scissors_dir = os.path.join('/tmp/rps/scissors')

print('total training rock images:', len(os.listdir(rock_dir)))
print('total training paper images:', len(os.listdir(paper_dir)))
print('total training scissors images:', len(os.listdir(scissors_dir)))


total training rock images: 840
total training paper images: 840
total training scissors images: 840


In [36]:
rock_files = os.listdir(rock_dir)
print(rock_files[:10])

paper_files = os.listdir(paper_dir)
print(paper_files[:10])

scissors_files = os.listdir(scissors_dir)
print(scissors_files[:10])

['rock06ck02-046.png', 'rock01-020.png', 'rock01-037.png', 'rock01-012.png', 'rock06ck02-053.png', 'rock03-073.png', 'rock06ck02-018.png', 'rock05ck01-034.png', 'rock07-k03-019.png', 'rock05ck01-102.png']
['paper06-013.png', 'paper03-066.png', 'paper07-063.png', 'paper03-031.png', 'paper07-115.png', 'paper03-058.png', 'paper01-091.png', 'paper01-043.png', 'paper01-097.png', 'paper06-085.png']
['scissors02-113.png', 'scissors04-069.png', 'scissors02-050.png', 'testscissors02-101.png', 'scissors03-002.png', 'scissors03-036.png', 'scissors03-091.png', 'scissors04-077.png', 'testscissors02-007.png', 'scissors01-089.png']


# Data Augmentation

Definimos las transformaciones a aplicar

Enlace con explicación de cada una de las transformaciones 
https://www.analyticsvidhya.com/blog/2020/08/image-augmentation-on-the-fly-using-keras-imagedatagenerator/

In [37]:
!pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "/tmp/rps/"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = "/tmp/rps-test-set/"
validation_datagen = ImageDataGenerator(rescale = 1./255)



Iterador para generar datos sobre directorios

Enlace donde se explican los parámetros: https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

In [39]:
train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


In [40]:
rock_dir = os.path.join('/tmp/rps/rock')
paper_dir = os.path.join('/tmp/rps/paper')
scissors_dir = os.path.join('/tmp/rps/scissors')

print('total training rock images:', len(os.listdir(rock_dir)))
print('total training paper images:', len(os.listdir(paper_dir)))
print('total training scissors images:', len(os.listdir(scissors_dir)))


total training rock images: 840
total training paper images: 840
total training scissors images: 840


# Creación del Modelo

Diseño del modelo

In [41]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    #tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])



model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 17, 17, 128)     

Compilación del Modelo, Entrenamiento del Modelo y Guardado de Pesos de los parámetros

In [42]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [43]:
history = model.fit(
    train_generator,
    epochs = 20,
    validation_data = validation_generator,
    verbose = 2)

Epoch 1/20
20/20 - 32s - loss: 2.4031 - accuracy: 0.3437 - val_loss: 1.0814 - val_accuracy: 0.3333 - 32s/epoch - 2s/step
Epoch 2/20
20/20 - 21s - loss: 1.0800 - accuracy: 0.4052 - val_loss: 1.0718 - val_accuracy: 0.3575 - 21s/epoch - 1s/step
Epoch 3/20
20/20 - 21s - loss: 1.1394 - accuracy: 0.4385 - val_loss: 0.9669 - val_accuracy: 0.6532 - 21s/epoch - 1s/step
Epoch 4/20
20/20 - 22s - loss: 0.9713 - accuracy: 0.5302 - val_loss: 0.9703 - val_accuracy: 0.5161 - 22s/epoch - 1s/step
Epoch 5/20
20/20 - 23s - loss: 0.9383 - accuracy: 0.5853 - val_loss: 0.5504 - val_accuracy: 0.7446 - 23s/epoch - 1s/step
Epoch 6/20
20/20 - 21s - loss: 0.7709 - accuracy: 0.6516 - val_loss: 0.5027 - val_accuracy: 0.8118 - 21s/epoch - 1s/step
Epoch 7/20
20/20 - 22s - loss: 0.6945 - accuracy: 0.6817 - val_loss: 0.7283 - val_accuracy: 0.6425 - 22s/epoch - 1s/step
Epoch 8/20
20/20 - 21s - loss: 0.6290 - accuracy: 0.7294 - val_loss: 0.9296 - val_accuracy: 0.5430 - 21s/epoch - 1s/step
Epoch 9/20
20/20 - 22s - loss: 0

Hacemos un grafico con la precisión de entrenamiento y validación:

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

# Uso del modelo para predecir imágenes

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)